In [ ]:
import json
import numpy as np
import pymatgen as pmg
import itertools
import pickle
import matplotlib.pyplot as plt
import glob

In [ ]:
elements = 'NbMoTaW'

In [ ]:
raw_data_training = []
raw_data_testing = []

files =  glob.glob('./raw/NbMoTaW/*.json')

test_files =  glob.glob('./raw/NbMoTaW/*Test.json')
train_files = list(set(files)-set(test_files))

for file in train_files:
    with open(file) as json_data:
        raw_data_training += [json.load(json_data)]

for file in test_files:
    with open(file) as json_data:
        raw_data_testing += [json.load(json_data)]

In [ ]:
train_files

In [ ]:
test_files

In [ ]:
#for generating test set

with open('raw/'+element+'/test.json') as json_data:
    raw_data_testing = json.load(json_data)


In [ ]:
#number of snapshots in each json file

len(raw_data_training)

In [ ]:
#for generating vacancy test set

#number of snapshots in each json file

len(raw_data_testing)

In [ ]:
#number of particles in each json file

(sum([len(i) for i in raw_data_training]))

In [ ]:
#number of particles in each json file

(set([len(i) for i in raw_data_training]))

In [ ]:
#for generating vacancy test set

#number of particles in each json file

(sum([len(i) for i in raw_data_testing]))

In [ ]:
data_all = raw_data_training

In [ ]:
#this is to generate my own split

import random

#generate test set indices

data_all_tosplit = raw_data_training
#data_all_tosplit = [data_npt,data_nvt,data_elastic,data_gb,data_surface,data_vacancy]

trainset_indices = [random.sample(range(i), i-(i//20)) for i in [len(j) for j in data_all_tosplit]]
data_mysplit_train = [data_all_tosplit[i][j] for i in range(len(data_all_tosplit)) for j in range(len(data_all_tosplit[i])) if j in trainset_indices[i]]

#the following is done to reduce training set size
temp_data = [ [j for ind_j,j in enumerate(i) if ind_j in trainset_indices[ind_i]] for ind_i,i in enumerate(data_all_tosplit)]
trainset_subset_indices = [random.sample(range(y), y//20) for y in [len(z) for z in temp_data]]


data_mysplit_train_subset = [temp_data[i][j] for i in range(len(temp_data)) for j in range(len(temp_data[i])) if j in trainset_subset_indices[i]]

data_mysplit_test = [data_all_tosplit[i][j] for i in range(len(data_all_tosplit)) for j in range(len(data_all_tosplit[i])) if j not in trainset_indices[i]]


In [ ]:
len(data_mysplit_train),len(data_mysplit_train_subset),len(data_mysplit_test)

In [ ]:
data_train_structure_energy = [(pmg.Structure.from_dict(i['structure']),i['outputs']['energy']/len(i['structure']['sites']),i['outputs']['forces']) 
                      for i in raw_data_training]

In [ ]:
#for generating test set

data_test_structure_energy = [(pmg.Structure.from_dict(i['structure']),i['outputs']['energy']/len(i['structure']['sites']),i['outputs']['forces']) 
                      for i in raw_data_testing]

In [ ]:
#this is to generate my own split

data_mysplit_train_structure_energy = [(pmg.Structure.from_dict(i['structure']),i['outputs']['energy']/i['num_atoms'],i['outputs']['forces']) 
                      for i in data_mysplit_train]

data_mysplit_test_structure_energy = [(pmg.Structure.from_dict(i['structure']),i['outputs']['energy']/i['num_atoms'],i['outputs']['forces'])
                      for i in data_mysplit_test]


In [ ]:
set_num_atoms = set([len(i['structure']['sites']) for i in raw_data_training])
numatoms_list_train = sorted([i for i in set_num_atoms])
numatoms_list_train

In [ ]:
with open('processed/'+elements+'/train/num_atoms_categories.pickle', 'wb') as handle:
    pickle.dump(numatoms_list_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#for generating test set

set_num_atoms_test = set([len(i['structure']['sites']) for i in raw_data_testing])


set_num_atoms = set_num_atoms_test
numatoms_list_test = sorted([i for i in set_num_atoms])
numatoms_list_test

In [ ]:
#for saving test categorized num

with open('processed/'+element+'/test/num_atoms_categories.pickle', 'wb') as handle:
    pickle.dump(numatoms_list_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#this is to generate my own split

set_num_atoms_mysplit_train = set([len(i['structure']['sites']) for i in data_mysplit_train])
set_num_atoms_mysplit_test = set([len(i['structure']['sites']) for i in data_mysplit_test])

set_num_atoms_mysplit_train = set_num_atoms_mysplit_train
numatoms_list_mysplit_train = sorted([i for i in set_num_atoms_mysplit_train])
numatoms_list_mysplit_train

In [ ]:
#this is to generate my own split

set_num_atoms_mysplit_test = set([len(i['structure']['sites']) for i in data_mysplit_test])

set_num_atoms_mysplit_test = set_num_atoms_mysplit_test
numatoms_list_mysplit_test = sorted([i for i in set_num_atoms_mysplit_test])
numatoms_list_mysplit_test

In [ ]:
with open('processed/'+elements+'/train/num_atoms_categories.pickle', 'wb') as handle:
    pickle.dump(numatoms_list_mysplit_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('processed/'+elements+'/test2/num_atoms_categories.pickle', 'wb') as handle:
    pickle.dump(numatoms_list_mysplit_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
data_train_struct_categorized = [[struct_energy[0] for struct_energy in data_train_structure_energy 
                                if len(struct_energy[0])==i] for i in numatoms_list_train]

data_train_struct_categorized_energy = [[struct_energy[1] for struct_energy in data_train_structure_energy 
                                        if len(struct_energy[0])==i] for i in numatoms_list_train]

data_train_struct_categorized_forces = [[struct_energy[2] for struct_energy in data_train_structure_energy 
                                        if len(struct_energy[0])==i] for i in numatoms_list_train]

In [ ]:
[len(i) for i in data_train_struct_categorized]

In [ ]:
#for generating test set

data_test_struct_categorized = [[struct_energy[0] for struct_energy in data_test_structure_energy 
                                if len(struct_energy[0])==i] for i in numatoms_list_test]

data_test_struct_categorized_energy = [[struct_energy[1] for struct_energy in data_test_structure_energy 
                                        if len(struct_energy[0])==i] for i in numatoms_list_test]

data_test_struct_categorized_forces = [[struct_energy[2] for struct_energy in data_test_structure_energy 
                                        if len(struct_energy[0])==i] for i in numatoms_list_test]

In [ ]:
[len(i) for i in data_test_struct_categorized]

In [ ]:
#this is to generate my own split

data_mysplit_train_struct_categorized = [[struct_energy[0] for struct_energy in data_mysplit_train_structure_energy 
                                if len(struct_energy[0])==i] for i in numatoms_list_mysplit_train]

data_mysplit_train_struct_categorized_energy = [[struct_energy[1] for struct_energy in data_mysplit_train_structure_energy 
                                        if len(struct_energy[0])==i] for i in numatoms_list_mysplit_train]

data_mysplit_train_struct_categorized_forces = [[struct_energy[2] for struct_energy in data_mysplit_train_structure_energy 
                                        if len(struct_energy[0])==i] for i in numatoms_list_mysplit_train]

In [ ]:
[len(i) for i in data_mysplit_train_struct_categorized]

In [ ]:
#this is to generate my own split

data_mysplit_test_struct_categorized = [[struct_energy[0] for struct_energy in data_mysplit_test_structure_energy 
                                if len(struct_energy[0])==i] for i in numatoms_list_mysplit_test]

data_mysplit_test_struct_categorized_energy = [[struct_energy[1] for struct_energy in data_mysplit_test_structure_energy 
                                        if len(struct_energy[0])==i] for i in numatoms_list_mysplit_test]

data_mysplit_test_struct_categorized_forces = [[struct_energy[2] for struct_energy in data_mysplit_test_structure_energy 
                                        if len(struct_energy[0])==i] for i in numatoms_list_mysplit_test]

In [ ]:
[len(i) for i in data_mysplit_test_struct_categorized]

In [ ]:
structs = [ j for i in data_mysplit_test_struct_categorized for j in i ]

In [ ]:
structs_neigh = [ neigh[1] for j in structs for atom in j.get_all_neighbors(5) for neigh in atom]

In [ ]:
with open('processed/'+element+'/train/categorized_structs.pickle', 'wb') as handle:
    pickle.dump(data_train_struct_categorized, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('processed/'+element+'/test2/categorized_structs.pickle', 'wb') as handle:
    pickle.dump(data_test_struct_categorized, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#this is to generate my own split

with open('processed/'+elements+'/train/categorized_structs.pickle', 'wb') as handle:
    pickle.dump(data_mysplit_train_struct_categorized, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#this is to generate my own split

with open('processed/'+elements+'/test2/categorized_structs.pickle', 'wb') as handle:
    pickle.dump(data_mysplit_test_struct_categorized, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('processed/'+element+'/train/categorized_energy.pickle', 'wb') as handle:
    pickle.dump(data_train_struct_categorized_energy, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#for generating test set

with open('processed/'+element+'/test/categorized_energy.pickle', 'wb') as handle:
    pickle.dump(data_test_struct_categorized_energy, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#this is to generate my own split

with open('processed/'+elements+'/train/categorized_energy.pickle', 'wb') as handle:
    pickle.dump(data_mysplit_train_struct_categorized_energy, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#this is to generate my own split

with open('processed/'+elements+'/test2/categorized_energy.pickle', 'wb') as handle:
    pickle.dump(data_mysplit_test_struct_categorized_energy, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('processed/'+element+'/train/categorized_forces.pickle', 'wb') as handle:
    pickle.dump(data_train_struct_categorized_forces, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#for generating test set

with open('processed/'+element+'/test/categorized_forces.pickle', 'wb') as handle:
    pickle.dump(data_test_struct_categorized_forces, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#this is to generate my own split

with open('processed/'+elements+'/train/categorized_forces.pickle', 'wb') as handle:
    pickle.dump(data_mysplit_train_struct_categorized_forces, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#this is to generate my own split

with open('processed/'+elements+'/test2/categorized_forces.pickle', 'wb') as handle:
    pickle.dump(data_mysplit_test_struct_categorized_forces, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
categorized_train_num_structs = [len(i) for i in data_train_struct_categorized]

In [ ]:
#for generating test set

categorized_test_num_structs = [len(i) for i in data_test_struct_categorized]